# Importing data

In [888]:
import pandas as pd

import warnings
import re
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [889]:
warnings.filterwarnings("ignore")

In [890]:
mobile_data = pd.read_csv("dataset/mobile_recommendation_system_dataset.csv")

In [891]:
mobile_data.head()

,name,ratings,price,imgURL,corpus
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...


# Checking for missing data and cleaning it

In [892]:
mobile_data.isnull().sum()

name        0
ratings     0
price       0
imgURL      0
corpus     12
dtype: int64

In [893]:
mobile_data.dropna(inplace=True)

In [894]:
mobile_data.duplicated().sum()

1

In [895]:
mobile_data.drop_duplicates(inplace=True)

In [896]:
mobile_data.shape

(2533, 5)

In [897]:
mobile_data["name"].duplicated().sum()

405

In [898]:
mobile_data.drop_duplicates(subset=["name"], inplace=True)

In [899]:
mobile_data.shape

(2128, 5)

In [900]:
# mobile_data["corpus"].duplicated().sum()

In [901]:
# mobile_data.drop_duplicates(subset=["corpus"], inplace=True)

In [902]:
# mobile_data.shape

# Creating required columns from the given dataset

In [903]:
mobile_data["corpus"][0]

'Storage128 GBRAM6  SystemAndroid 12Processor TypeMediatek Dimensity 1080Processor Speed2.6 50MP 8MP 2MP 50MP 16MP 5G Capacity5000 Display Size16.94 cm (6.67 inch)Resolution2400 x 1080 PixelsResolution TypeFull HD+ AMOLED DisplayGPUARM Mali-G68 MC4Display TypeFull HD+ AMOLED DisplayOther Display FeaturesRefresh Rate: 120 Hz, Contrast: 5,000,000:1, 1920Hz PWM Dimming, Brightness Level: 16000:1, Peak Brightness: 900 nits'

In [904]:
# creating storage column
def create_storage_column(corpus):
    query = re.search(r"Storage(\d+) ", corpus)

    if query:
        return query.group(1).strip()
    return "Other"

mobile_data["Storage"] = mobile_data["corpus"].apply(create_storage_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128


In [905]:
# creating ram column
def create_ram_column(corpus):
    query = re.search(r"RAM(\d+)", corpus)

    if query:
        return query.group(1).strip()
    return "Other"

mobile_data["RAM"] = mobile_data["corpus"].apply(create_ram_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other


In [906]:
# creating system column
def create_system_column(corpus):
    query = re.search(r"System(.*?)Processor", corpus)

    if query:
        return query.group(1).strip()
    return "Other"

mobile_data["System"] = mobile_data["corpus"].apply(create_system_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM,System
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6,Android 12
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6,Android Pie 9.0
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4,Android 11
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12,Android Q 11
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other,iOS 15


In [907]:
# creating os name column
def create_os_name_column(system):
    query = re.search(r"(.*?) ", system)

    if query:
        return query.group(1).strip()
    return "Other"

mobile_data["OS_Name"] = mobile_data["System"].apply(create_os_name_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM,System,OS_Name
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6,Android 12,Android
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6,Android Pie 9.0,Android
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4,Android 11,Android
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12,Android Q 11,Android
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other,iOS 15,iOS


In [908]:
# creating os version column
def creat_os_version_column(system):
    query = re.search(r" (\d+)", system)

    if query:
        return query.group(1).strip()
    return "Other"

mobile_data["OS_Version"] = mobile_data["System"].apply(creat_os_version_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM,System,OS_Name,OS_Version
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6,Android 12,Android,12
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6,Android Pie 9.0,Android,9
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4,Android 11,Android,11
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12,Android Q 11,Android,11
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other,iOS 15,iOS,15


In [909]:
# create brand column
def create_brand_column(name):
    query = re.search(r"(.*?) ", name)

    if query:
        return query.group(1).title().strip()
    return "Other"

mobile_data["Brand"] = mobile_data["name"].apply(create_brand_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM,System,OS_Name,OS_Version,Brand
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6,Android 12,Android,12,Redmi
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6,Android Pie 9.0,Android,9,Oppo
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4,Android 11,Android,11,Redmi
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12,Android Q 11,Android,11,Oneplus
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other,iOS 15,iOS,15,Apple


In [910]:
# creating processor column
def create_processor_column(corpus):
    query = re.search(r"Processor Type(.*?) ", corpus)

    if query:
        return query.group(1).strip()
    return "Other"

mobile_data["Processor"] = mobile_data["corpus"].apply(create_processor_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM,System,OS_Name,OS_Version,Brand,Processor
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6,Android 12,Android,12,Redmi,Mediatek
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6,Android Pie 9.0,Android,9,Oppo,MediaTek
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4,Android 11,Android,11,Redmi,Other
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12,Android Q 11,Android,11,Oneplus,Qualcomm
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other,iOS 15,iOS,15,Apple,A15


In [911]:
# creating processor speed column
def create_processor_speed_column(corpus):
    query = re.search(r"Speed(.*?) ", corpus)

    if query:
        return query.group(1).strip()
    return "Other"

mobile_data["Processor_Speed"] = mobile_data["corpus"].apply(create_processor_speed_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM,System,OS_Name,OS_Version,Brand,Processor,Processor_Speed
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6,Android 12,Android,12,Redmi,Mediatek,2.6
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6,Android Pie 9.0,Android,9,Oppo,MediaTek,2.1
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4,Android 11,Android,11,Redmi,Other,2.4
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12,Android Q 11,Android,11,Oneplus,Qualcomm,2.4
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other,iOS 15,iOS,15,Apple,A15,Other


In [912]:
# creating network column
def create_network_column(corpus):
    query = re.search(r" (\d)G ", corpus)

    if query:
        return query.group(1).strip()
    return "Other"

mobile_data["Network"] = mobile_data["corpus"].apply(create_network_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM,System,OS_Name,OS_Version,Brand,Processor,Processor_Speed,Network
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6,Android 12,Android,12,Redmi,Mediatek,2.6,5
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6,Android Pie 9.0,Android,9,Oppo,MediaTek,2.1,3
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4,Android 11,Android,11,Redmi,Other,2.4,4
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12,Android Q 11,Android,11,Oneplus,Qualcomm,2.4,5
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other,iOS 15,iOS,15,Apple,A15,Other,5


In [913]:
# creating resolution column
def create_resolution_column(corpus):
    query = re.search(r"Resolution(\d+ x \d+) ", corpus)

    if query:
        return query.group(1).strip()
    return "Other"

mobile_data["Resolution"] = mobile_data["corpus"].apply(create_resolution_column)

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM,System,OS_Name,OS_Version,Brand,Processor,Processor_Speed,Network,Resolution
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6,Android 12,Android,12,Redmi,Mediatek,2.6,5,2400 x 1080
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6,Android Pie 9.0,Android,9,Oppo,MediaTek,2.1,3,2340 x 1080
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4,Android 11,Android,11,Redmi,Other,2.4,4,2400 x 1080
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12,Android Q 11,Android,11,Oneplus,Qualcomm,2.4,5,Other
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other,iOS 15,iOS,15,Apple,A15,Other,5,2340 x 1080


In [914]:
mobile_data.isnull().sum()

name               0
ratings            0
price              0
imgURL             0
corpus             0
Storage            0
RAM                0
System             0
OS_Name            0
OS_Version         0
Brand              0
Processor          0
Processor_Speed    0
Network            0
Resolution         0
dtype: int64

In [915]:
mobile_data.duplicated().sum()

0

In [916]:
mobile_data.columns

Index(['name', 'ratings', 'price', 'imgURL', 'corpus', 'Storage', 'RAM',
       'System', 'OS_Name', 'OS_Version', 'Brand', 'Processor',
       'Processor_Speed', 'Network', 'Resolution'],
      dtype='object')

In [917]:
mobile_data["tags"] = mobile_data["name"].str.cat(mobile_data["Storage"], sep=", ").str.cat(mobile_data["RAM"], sep=", ").str.cat(mobile_data["System"], sep=", ").str.cat(mobile_data["OS_Name"], sep=", ").str.cat(mobile_data["OS_Version"], sep=", ").str.cat(mobile_data["Brand"], sep=", ").str.cat(mobile_data["Processor"], sep=", ")

mobile_data.head()

,name,ratings,price,imgURL,corpus,Storage,RAM,System,OS_Name,OS_Version,Brand,Processor,Processor_Speed,Network,Resolution,tags
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,128,6,Android 12,Android,12,Redmi,Mediatek,2.6,5,2400 x 1080,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB), 128..."
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,128,6,Android Pie 9.0,Android,9,Oppo,MediaTek,2.1,3,2340 x 1080,"OPPO F11 Pro (Aurora Green, 128 GB), 128, 6, A..."
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,64,4,Android 11,Android,11,Redmi,Other,2.4,4,2400 x 1080,"REDMI Note 11 (Starburst White, 64 GB), 64, 4,..."
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,256,12,Android Q 11,Android,11,Oneplus,Qualcomm,2.4,5,Other,"OnePlus Nord CE 5G (Blue Void, 256 GB), 256, 1..."
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,128,Other,iOS 15,iOS,15,Apple,A15,Other,5,2340 x 1080,"APPLE iPhone 13 mini (Blue, 128 GB), 128, Othe..."


# Main Recommendation System Logic

In [958]:
count_vectorizer = CountVectorizer(max_features=1000, stop_words="english")

In [959]:
count_vectorizer

CountVectorizer(max_features=1000, stop_words='english')

In [960]:
vectors = count_vectorizer.fit_transform(mobile_data["corpus"]).toarray()

In [961]:
vectors

array([[0, 2, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [962]:
count_vectorizer.get_feature_names_out()

array(['00', '000', '02', '03', '04', '05', '05mm', '07', '07bother',
       '08', '09', '0primary', '0processor', '10', '100', '1000',
       '100000', '1000000', '100mp', '103', '104', '106', '1080',
       '1080processor', '1080resolution', '108mp', '10mp', '10processor',
       '11', '1100', '1125', '1170', '1179', '11primary', '11processor',
       '12', '120', '1200', '1200processor', '120hz', '1242', '1284',
       '1290', '12mp', '12primary', '12processor', '13', '1300',
       '1300processor', '1330', '1334', '135', '1380', '13mp',
       '13processor', '14', '1400', '144', '1440', '144hz', '144hzother',
       '1480', '14processor', '15', '1500', '1520', '1544', '15processor',
       '16', '1600', '16000', '1600resolution', '1612', '1640', '16mp',
       '16processor', '18', '180', '180hz', '19', '1920', '1920hz',
       '1processor', '20', '200', '2000000', '200mp', '20mp', '21', '22',
       '23', '2340', '24', '240', '2400', '2408', '240hz', '2460', '24mp',
       '25mm', 

In [963]:
similarity = cosine_similarity(vectors)

In [964]:
similarity

array([[1.        , 0.24692035, 0.35556531, ..., 0.49706683, 0.32593487,
        0.36465258],
       [0.24692035, 1.        , 0.15686275, ..., 0.26314581, 0.19607843,
        0.37919764],
       [0.35556531, 0.15686275, 1.        , ..., 0.28194194, 0.47058824,
        0.28439823],
       ...,
       [0.49706683, 0.26314581, 0.28194194, ..., 1.        , 0.25374774,
        0.44611344],
       [0.32593487, 0.19607843, 0.47058824, ..., 0.25374774, 1.        ,
        0.23268946],
       [0.36465258, 0.37919764, 0.28439823, ..., 0.44611344, 0.23268946,
        1.        ]])

In [965]:
def recommend_mobiles(mobile):
    mobile_index = mobile_data[mobile_data["name"] == mobile].index[0]
    distances = similarity[mobile_index]
    mobile_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in mobile_list:
        print(mobile_data["name"].iloc[i[0]])

In [966]:
recommend_mobiles("REDMI Note 12 Pro 5G (Onyx Black, 128 GB)")

REDMI Note 12 Pro 5G (Stardust Purple, 128 GB)
REDMI Note 12 Pro 5G (Glacier Blue, 128 GB)
REDMI Note 12 Pro 5G (Onyx Black, 256 GB)
REDMI Note 12 Pro 5G (Stardust Purple, 256 GB)
REDMI Note 12 Pro 5G (Glacier Blue, 256 GB)


In [967]:
recommend_mobiles("APPLE iPhone 13 mini (Blue, 128 GB)")

APPLE iPhone 13 mini ((PRODUCT)RED, 128 GB)
APPLE iPhone 13 mini (Starlight, 128 GB)
APPLE iPhone 13 mini (Green, 128 GB)
APPLE iPhone 13 mini (Midnight, 128 GB)
APPLE iPhone 13 mini (Green, 256 GB)


In [968]:
recommend_mobiles("Redmi 4A (Dark Grey, 32 GB)")

OPPO F7 (Black, 64 GB)
OPPO Hardik Pandya Limited Editon (Black, 64 GB)
OPPO F7 (Silver, 64 GB)
OPPO F7 (Red, 64 GB)
OPPO F7 (Red, 128 GB)


# Exporting data into pickle files

In [929]:
pickle.dump(mobile_data.to_dict(), open("mobiles.pkl", "wb"))

In [930]:
pickle.dump(similarity, open("similarity.pkl", "wb"))